In [1]:
#This is for Google Collab Notebook - it doesn't have torchplot installed!
!pip install torchplot
# Here we are importing the necessary libraries
import numpy as np
import torch
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import torchplot as tplt
import numpy as np
import time
import os
import copy
import torch.optim as optim

from torch import nn
from torch import optim
from torchvision import datasets, models, transforms
from tqdm import tqdm
from torch.utils.data import DataLoader, sampler, random_split

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We will be using the:", device)

In [2]:
transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize(227),
            transforms.CenterCrop(227),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Our two datasets
emnist_dataset = datasets.EMNIST(root='./', split='balanced', train=True, download=True, transform=transform)
math_dataset   = datasets.ImageFolder('../input/handwritten-math-symbols/dataset', transform=transform)

# Load dataset
complete_dataset = torch.utils.data.ConcatDataset([math_dataset, emnist_dataset])
train_dataset = DataLoader(complete_dataset,batch_size=10,)

In [3]:
data = iter(train_dataset) # Let's iterate on it
single_point = next(data)
ToPIL = transforms.ToPILImage() # Converting function
img0 = ToPIL(single_point[0][0])
img1 = ToPIL(single_point[0][1])
# Plotting
fig, axs = plt.subplots(1,2)
axs[0].imshow(img0)
axs[1].imshow(img1)

In [4]:
class LeNet(nn.Module):
    def __init__(self, nclasses=68):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6,5,padding=2)
        self.conv2 = nn.Conv2d(6,16,5, padding=0)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,nclasses)
        
    def forward(self, x):
        x = torch.sigmoid(self.conv1(x))
        x =  nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = torch.sigmoid(self.conv2(x))
        x =  nn.MaxPool2d(kernel_size=2, stride=2)(x)
        
        x = x.view(-1, 5*5*16)
        
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
# Code from https://medium.com/analytics-vidhya/alexnet-a-simple-implementation-using-pytorch-30c14e8b6db2
class AlexNet(nn.Module):
    def __init__(self, nclasses=68):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 96, kernel_size= 11, stride=4, padding=0 )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride= 1, padding= 2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride= 1, padding= 1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.fc1  = nn.Linear(in_features= 9216, out_features= 4096)
        self.fc2  = nn.Linear(in_features= 4096, out_features= 4096)
        self.fc3 = nn.Linear(in_features=4096 , out_features=nclasses)


    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
# Will append the test accuracy to the test_accuracy_history[]
def evaluate(model, test_data, test_accuracy_history, test_image_datasets_len):
    with torch.no_grad():
        test_accuracy = 0
        for (data, label) in test_data:
            data = data.to(device)
            label = label.to(device)
            out = model(data)
            answers = out.max(dim=1)[1]
            test_accuracy += (answers == label).sum()
    # Append the testing accuracy
    test_accuracy = test_accuracy / test_image_datasets_len * 100
    test_accuracy_history.append(test_accuracy)
    return test_accuracy
# Does not accept the test_accuracy_history[]
def evaluateAcc(model, test_data, test_data_len):
    with torch.no_grad():
        test_accuracy = 0
        for (data, label) in test_data:
            data = data.to(device)
            label = label.to(device)
            out = model(data)
            answers = out.max(dim=1)[1]
            test_accuracy += (answers == label).sum()
    # Append the testing accuracy
    test_accuracy = test_accuracy / test_data_len * 100
    return test_accuracy

In [7]:
def super_plotter(loss_history, test_accuracy_history, train_accuracy_history):
    # Plot the loss per iteration
    tplt.plot(loss_history, label="Loss")
    tplt.title("Neural Network Loss")
    tplt.xlabel("Number of iterations")
    tplt.ylabel("Loss History")
    tplt.legend(loc='upper right')
    tplt.show()
    # Plot the accuracy per epoch
    tplt.plot(test_accuracy_history, label="Test")
    tplt.plot(train_accuracy_history, label="Train")
    tplt.title("Neural Network Accuracy")
    tplt.xlabel("Epoch Number")
    tplt.ylabel("Accuracy")
    tplt.legend(loc='lower right')
    tplt.show()

In [8]:
def super_model_trainer(model, train_dataset, train_dataloader, test_dataset, test_dataloader, batch_size, optimizer, nepochs): 
    model.train()
    loss_history = []
    test_accuracy_history = []
    train_accuracy_history = []
    loss = torch.Tensor([0])
    for epoch in tqdm(range(nepochs), desc=f"Epoch", unit="epoch", disable=False):
        accuracy = 0
        for (data, label) in tqdm(train_dataloader, desc="iteration", unit="%", disable=True):
            # Here we clear the gradients
            optimizer.zero_grad(set_to_none=True)        
            # We need to make sure the tensors are on the same device as our model
            data = data.to(device)
            label = label.to(device)
            out = model(data)
        
            loss = LossFunction(out, label)
        
            loss.backward() # This function calculates all our gradients
            optimizer.step() # This function does our gradient descent with those gradients
            loss_history.append(loss.item())
            answers = out.max(dim=1)[1]
            accuracy += (answers == label).sum()
        # Append the training accuracy
        accuracy = accuracy / len(train_dataset)*100
        train_accuracy_history.append(accuracy)
    
        #let's get the test accuracy to see how well it generalizes
        test_accuracy = evaluate(model, test_dataloader, test_accuracy_history, len(test_dataset))
        #Print the results
        print(f"Epoch: {epoch} \n Loss: {loss.item()} \n Train Accuracy: {accuracy:.2f}% \n Test Accuracy: {test_accuracy:.2f}%")
    super_plotter(loss_history, test_accuracy_history, train_accuracy_history)

In [ ]:
# Let's set up some parameters
learning_rate=0.001
nepochs = 5
nclasses=53
batch_size = 64


#model = LeNet(nclasses).to(device)
model = AlexNet(nclasses).to(device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), learning_rate)
LossFunction = nn.CrossEntropyLoss()

transformAlexNet = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize(227),
            transforms.CenterCrop(227),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
transformLeNet = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize(28),
            transforms.CenterCrop(28),
            transforms.ToTensor(),
            #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

transform = transformAlexNet

# Create the emnist train and test datasets
emnist_train_dataset = datasets.EMNIST(root='./', split='balanced', train=True, download=True, transform=transform)
emnist_test_dataset = datasets.EMNIST(root='./', split='balanced', train=False, download=False, transform=transform)

# Create the math train and test datasets
math_dataset = datasets.ImageFolder('../input/customarchive/CustomArchive/dataset', transform=transform)
#math_dataset = datasets.ImageFolder('./CustomArchive/dataset', transform=transform)

math_train_dataset_len = int(len(math_dataset)*0.80)
math_test_dataset_len = int(len(math_dataset) - math_train_dataset_len)
math_train_dataset, math_test_dataset = random_split(math_dataset, [math_train_dataset_len, math_test_dataset_len])

# Combine the train and test datasets and shuffle them together
train_dataset = torch.utils.data.ConcatDataset([math_train_dataset, emnist_train_dataset])
train_dataloader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_dataset = torch.utils.data.ConcatDataset([math_test_dataset, emnist_test_dataset])
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

#Training 
super_model_trainer(model, train_dataset, train_dataloader, test_dataset, test_dataloader, batch_size, optimizer, nepochs)

In [ ]:
test_data_emnist = DataLoader(emnist_test_dataset,batch_size=batch_size,shuffle=True)
test_data_math = DataLoader(math_test_dataset,batch_size=batch_size, shuffle=True)


test_accuracy_emnist = evaluateAcc(model, test_data_emnist, len(emnist_test_dataset))
test_accuracy_math = evaluateAcc(model, test_data_math, math_test_dataset_len)



m_dataset = datasets.ImageFolder('../input/handwritten-math-symbols/dataset', transform=transform)
m_dataloader = DataLoader(m_dataset,batch_size=batch_size,shuffle=True)
m_accuracy = evaluateAcc(model, m_dataloader, len(m_dataset))

#Print the results
print(f"EMNIST Accuracy: {test_accuracy_emnist:.2f}%")
print(f"Operator Accuracy: {test_accuracy_math:.2f}%")
print(f"Full Archive Accuracy: {m_accuracy:.2f}%")